In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as BS
import numpy as np
import re
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import seaborn as sns
import urllib.robotparser as urobot
from urllib.request import urlopen
from pylab import rcParams
import csv

%matplotlib inline

In [2]:
#Load both list_of_champs table & prospects'

df1 = pd.read_csv('../data/list_of_champs.csv')
df2 = pd.read_csv('../data/Top50_prospects.csv')
df2 = df2.dropna(axis=1)
df2

,Rank,Name,Weight Class,Record,Age,Fighting Style,Company,Country,Nickname
0,1,Vadim Nemkov,Light Heavyweight,11-2,27,Wrestling,Bellator,Russia,Unknown
1,2,Lance Palmer,Featherweight,22-3,32,Wrestling,PFL,USA,Unknown
2,3,A.J. McKee,Featherweight,16-0,25,Wrestling,Bellator,USA,Unknown
3,4,Natan Schulte,Lightweight,20-3-1,28,Jiu-Jitsu,PFL,Brazil,Russo
4,5,Darrion Caldwell,Featherweight,15-3,32,Wrestling,Bellator,USA,Unknown
5,6,Jiri Prochazka,Light Heavyweight,26-3-1,27,Boxing,UFC,Czech republic,Unknown
6,7,Valentin Moldavsky,Heavyweight,10-1,28,"Sambo, MMA",Bellator,Russia,Unknown
7,8,Abdul-Aziz Abdulvakhabov,Lightweight,18-2,31,Wrestling,ACA,Russia,Lion
8,9,Tomasz Narkun,Light Heavyweight,18-3,30,Jiu-Jitsu,KSW,Poland,Giraffe
9,10,Pedro Carvalho,Featherweight,11-4,24,Jiu-Jitsu,Bellator,Portugal,The Game


In [3]:
###Adjust df2 to match up with the list of champs before joining both tables###

df2 = pd.read_csv('../data/Top50_prospects.csv', usecols = [' Weight Class',
                                                            ' Name',
                                                            ' Record',
                                                            'Fighting Style',
                                                            'Country',
                                                            'Nickname'])
df2

,Name,Weight Class,Record,Fighting Style,Country,Nickname
0,Vadim Nemkov,Light Heavyweight,11-2,Wrestling,Russia,Unknown
1,Lance Palmer,Featherweight,22-3,Wrestling,USA,Unknown
2,A.J. McKee,Featherweight,16-0,Wrestling,USA,Unknown
3,Natan Schulte,Lightweight,20-3-1,Jiu-Jitsu,Brazil,Russo
4,Darrion Caldwell,Featherweight,15-3,Wrestling,USA,Unknown
5,Jiri Prochazka,Light Heavyweight,26-3-1,Boxing,Czech republic,Unknown
6,Valentin Moldavsky,Heavyweight,10-1,"Sambo, MMA",Russia,Unknown
7,Abdul-Aziz Abdulvakhabov,Lightweight,18-2,Wrestling,Russia,Lion
8,Tomasz Narkun,Light Heavyweight,18-3,Jiu-Jitsu,Poland,Giraffe
9,Pedro Carvalho,Featherweight,11-4,Jiu-Jitsu,Portugal,The Game


In [4]:
## Since df2 only showcases the top50 'MALE' MMA prospects, I'll add a 'Sex' colummn to clarify my analysis later on btw male prospects vs champs

df2['Sex'] = 'Male'
df2

,Name,Weight Class,Record,Fighting Style,Country,Nickname,Sex
0,Vadim Nemkov,Light Heavyweight,11-2,Wrestling,Russia,Unknown,Male
1,Lance Palmer,Featherweight,22-3,Wrestling,USA,Unknown,Male
2,A.J. McKee,Featherweight,16-0,Wrestling,USA,Unknown,Male
3,Natan Schulte,Lightweight,20-3-1,Jiu-Jitsu,Brazil,Russo,Male
4,Darrion Caldwell,Featherweight,15-3,Wrestling,USA,Unknown,Male
5,Jiri Prochazka,Light Heavyweight,26-3-1,Boxing,Czech republic,Unknown,Male
6,Valentin Moldavsky,Heavyweight,10-1,"Sambo, MMA",Russia,Unknown,Male
7,Abdul-Aziz Abdulvakhabov,Lightweight,18-2,Wrestling,Russia,Lion,Male
8,Tomasz Narkun,Light Heavyweight,18-3,Jiu-Jitsu,Poland,Giraffe,Male
9,Pedro Carvalho,Featherweight,11-4,Jiu-Jitsu,Portugal,The Game,Male


In [5]:
## Now, let's merge both tables = combined list

df3 = pd.merge(df1, df2, how = 'outer')
df3

,Weight Class,Name,Sex,Record,Fighting Style,Country,Nickname,Title Holder
0,Heavyweight\n,Mark Coleman\n,Male,16-10,Wrestling,USA,The Hammer,Former
1,Heavyweight\n,Maurice Smith\n,Male,14-14,Kickboxing,USA,Mo,Former
2,Heavyweight\n,Randy Couture\n,Male,19-11,Wrestling,USA,The Natural,Former
3,Heavyweight\n,Randy Couture\n,Male,19-11,Wrestling,USA,The Natural,Former
4,Heavyweight\n,Bas Rutten\n,Male,28-4,"Kickboxing, Wrestling",Netherlands,El Guapo (The Handsome One),Former
...,...,...,...,...,...,...,...,...
164,Light Heavyweight,Nasrudin Nasrudinov,Male,9-0,Wrestling,Russia,Unknown,NaN
165,Middleweight,Vladimir Mineev,Male,13-1-1,Kickboxing,Russia,Unknown,NaN
166,Bantamweight,Patrick Mix,Male,13-0,"Wrestling, Jiu-Jitsu",USA,No Love,NaN
167,Lightweight,Soren Bak,Male,13-1,"Wrestling, Thai Boxing",Unknown,The True Viking,NaN


In [6]:
df3.tail()

,Weight Class,Name,Sex,Record,Fighting Style,Country,Nickname,Title Holder
164,Light Heavyweight,Nasrudin Nasrudinov,Male,9-0,Wrestling,Russia,Unknown,NaN
165,Middleweight,Vladimir Mineev,Male,13-1-1,Kickboxing,Russia,Unknown,NaN
166,Bantamweight,Patrick Mix,Male,13-0,"Wrestling, Jiu-Jitsu",USA,No Love,NaN
167,Lightweight,Soren Bak,Male,13-1,"Wrestling, Thai Boxing",Unknown,The True Viking,NaN
168,Lightweight,Khusein Khaliev,Male,19-2,Wrestling,Russia,Captain,NaN


In [7]:
# Remove "\n" on both weight class, name columns

df3[' Weight Class'] = df3[' Weight Class'].str.strip()
df3.head()

,Weight Class,Name,Sex,Record,Fighting Style,Country,Nickname,Title Holder
0,Heavyweight,Mark Coleman\n,Male,16-10,Wrestling,USA,The Hammer,Former
1,Heavyweight,Maurice Smith\n,Male,14-14,Kickboxing,USA,Mo,Former
2,Heavyweight,Randy Couture\n,Male,19-11,Wrestling,USA,The Natural,Former
3,Heavyweight,Randy Couture\n,Male,19-11,Wrestling,USA,The Natural,Former
4,Heavyweight,Bas Rutten\n,Male,28-4,"Kickboxing, Wrestling",Netherlands,El Guapo (The Handsome One),Former


In [8]:
df3[' Name'] = df3[' Name'].str.strip()
df3.head()

,Weight Class,Name,Sex,Record,Fighting Style,Country,Nickname,Title Holder
0,Heavyweight,Mark Coleman,Male,16-10,Wrestling,USA,The Hammer,Former
1,Heavyweight,Maurice Smith,Male,14-14,Kickboxing,USA,Mo,Former
2,Heavyweight,Randy Couture,Male,19-11,Wrestling,USA,The Natural,Former
3,Heavyweight,Randy Couture,Male,19-11,Wrestling,USA,The Natural,Former
4,Heavyweight,Bas Rutten,Male,28-4,"Kickboxing, Wrestling",Netherlands,El Guapo (The Handsome One),Former


In [9]:
#Let's split the record column into two more descriptive 'Wins' & 'Losses' columns

split = lambda x:pd.Series([i for i in (x.split('-'))])

In [10]:
Record_df = df3[' Record'].apply(split)
Record_df

,0,1,2
0,16,10,NaN
1,14,14,NaN
2,19,11,NaN
3,19,11,NaN
4,28,4,NaN
...,...,...,...
164,9,0,NaN
165,13,1,1
166,13,0,NaN
167,13,1,NaN


In [11]:
Record_df.columns = ['Wins',
                     'Losses',
                     'Draws']

Record_df.head(2)

,Wins,Losses,Draws
0,16,10,NaN
1,14,14,NaN


In [12]:
Record_df.drop(['Draws'], axis=1)

,Wins,Losses
0,16,10
1,14,14
2,19,11
3,19,11
4,28,4
...,...,...
164,9,0
165,13,1
166,13,0
167,13,1


In [13]:
#split fighting styles in multiple columns

split = lambda x:pd.Series([i for i in (x.split('-'))])
FS_df = df1['Fighting Style'].apply(split)
FS_df

,0,1,2
0,Wrestling,NaN,NaN
1,Kickboxing,NaN,NaN
2,Wrestling,NaN,NaN
3,"Kickboxing, Wrestling",NaN,NaN
4,"Wrestling, Boxing",NaN,NaN
...,...,...,...
114,"Wrestling, Jiu",Jitsu,NaN
115,"Muay thai, Kickboxing",NaN,NaN
116,Jiu,"Jitsu, Karate, Taekwondo",NaN
117,Jiu,"Jitsu, Boxing, Muat thai",NaN


In [14]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 169 entries, 0 to 168
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0    Weight Class   169 non-null    object
 1    Name           169 non-null    object
 2   Sex             169 non-null    object
 3    Record         169 non-null    object
 4   Fighting Style  169 non-null    object
 5   Country         167 non-null    object
 6   Nickname        169 non-null    object
 7   Title Holder    119 non-null    object
dtypes: object(8)
memory usage: 11.9+ KB


In [15]:
df3.describe

<bound method NDFrame.describe of           Weight Class                 Name   Sex   Record  \
0          Heavyweight         Mark Coleman  Male    16-10   
1          Heavyweight        Maurice Smith  Male    14-14   
2          Heavyweight        Randy Couture  Male    19-11   
3          Heavyweight        Randy Couture  Male    19-11   
4          Heavyweight           Bas Rutten  Male    28-4    
..                 ...                  ...   ...      ...   
164  Light Heavyweight  Nasrudin Nasrudinov  Male      9-0   
165       Middleweight      Vladimir Mineev  Male   13-1-1   
166       Bantamweight          Patrick Mix  Male     13-0   
167        Lightweight            Soren Bak  Male     13-1   
168        Lightweight      Khusein Khaliev  Male     19-2   

             Fighting Style      Country                     Nickname  \
0                 Wrestling          USA                   The Hammer   
1                Kickboxing          USA                           Mo   
2 

In [16]:
df3.dtypes

 Weight Class     object
 Name             object
Sex               object
 Record           object
Fighting Style    object
Country           object
Nickname          object
Title Holder      object
dtype: object

#Count of current and former champs

df3 = df1.groupby(['Title Holder']).agg(['count'])
df3 = df1.drop([' Name','Sex','Fighting Style','Country','Nickname'], axis=1)
df3.head()

Current_champs = df3[df3['Title Holder'] == 'Current']
Current_champs

Current_champs['Title Holder'].value_counts